In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

# Iterators for array-based lists

Iterators were described in some detail in the [Interfaces notebook](../part3/iterfaces.ipynb#notebook_id). Recall that
[Iterator](https://docs.oracle.com/javase/8/docs/api/java/util/Iterator.html) is an interface that describes the behaviour of an object that can iterate
over the elements of a collection or sequence; optionally, an iterator can remove an element of the collection as it iterates over the elements.

There are
four methods in the interface where two of the methods are `default` methods. For lists, we usually want to override the implementation
of the default method `remove`:

```java
package java.util;

public interface Iterator<T> {
    
    public boolean hasNext();
    public T next();
    public default void remove() {
         throw new UnsupportedOperationException();
    }
    
    // one more default method not shown here
}
```

`hasNext` returns `true` if there is another element in the collection or sequence that has not yet been visited by the iterator.

`next` returns the next element in the collection or sequence for this iterator, or throws a `NoSuchElementException` if there are no more
elements to be visited in the collection.

`remove` is an optional operation for an iterator, and its default implementation is to throw an `UnsupportedOperationException`.
If `remove` is overridden by a class, then
`remove` removes the element that was most recently returned by `next` and can be called only once for each call to `next`.
An `IllegalStateException` is thrown if the `next` method has not yet been called, or the `remove` method has already been called after the last call to the `next` method.

As desribed in the [Lists](../part1/lists.ipynb#notebook_id) and [Sets](./part1/sets.ipynb#notebook_id) notebooks, the only safe way to remove elements from the collection
while iterating over the collection is to use an iterator; for example, the following loop removes all negative values from a list:

```java
// assume t refers to a List
for (Iterator<Integer> iter = t.iterator(); iter.hasNext(); ) {
    Integer val = iter.next();
    if (val < 0) {
        iter.remove();    // can call remove once for each call to next
    }
}
```

## An iterator is similar to a cursor

An iterator is similar to a cursor in a text editor where the cursor sits *between* letters. The figure below shows part of the screen
of a Java text editor where the blinking cursor is between the letters `y` and `I`:

![](../resources/images/array_list/iterator_as_cursor.gif)

An iterator over a list sits between elements in the list. The figure below shows a list having 15 elements and an iterator in red between the two
elements `13` and `6`:

![](../resources/images/array_list/iterator_1.png)

In the figure above, the iterator is somewhere in the middle of the list so it has an element immediately before and after it current
location; we will refer to these elements as the *previous* and *next* elements.

If an iterator has a next element, then the `hasNext` method will return `true` indicating that there are more elements available
to iterate over. In the figure above, calling `hasNext` returns `true` because the iterator has the next element `6`.

If an iterator has a next element, then the `next` method will return the next element and advance the iterator one position.
In the figure above, calling `next` returns the element `6` and advances the iterator one position to the right so that
it sits between the elements `6` and `10` as shown in the following figure:

![](../resources/images/array_list/iterator_2.png)

If `next` is called six more times, then the iterator reaches a position where its previous element is the last element in the list
(`11`) and its next element does not exist as shown in the following figure:

![](../resources/images/array_list/iterator_3.png)

In the figure above, calling `hasNext` returns `false` and calling `next` results in a `NoSuchElementException` being thrown.

A list iterator starting at the beginning of the list has no previous element as shown in the following figure:

![](../resources/images/array_list/iterator_4.png)

## `remove`

`remove` removes the element that was most recently returned by `next` and can be called only once for each call to `next`.
An `IllegalStateException` is thrown if the `next` method has not yet been called, or the `remove` method has already been called after the last call to the `next` method.

The following figure shows an iterator between the elements `13` and `6`:

![](../resources/images/array_list/iterator_remove_1.png)

Calling `next` on the iterator returns the `6` and advances the iterator:

![](../resources/images/array_list/iterator_remove_2.png)

Calling `remove` on the iterator removes the element that was most returned by `next`; this causes the `6` to be removed from the list which modifies
the list as shown in the following figure:

![](../resources/images/array_list/iterator_remove_3.png)

Notice that calling `next` on the iterator should return the element `10` but because an element was removed the next element for the iterator shown
in the figure above is the element `4`. The `remove` method must address this issue by moving the iterator back one position before the method completes.
Moving the iterator back one position moves the iterator to its correct position as shown in the following figure:

![](../resources/images/array_list/iterator_remove_4.png)


## Implementing an iterator for an array-based list

The `ArrayIterator` class is an iterator for an array-based list. It is defined as a private inner class of the `SArrayList` class. 
In the discussion above, an iterator was described as having previous and next elements. Rather than storing references to the previous and next elements,
it simplifies the implementation to store the *indexes* of the two elements that the iterator is currently between. An `ArrayIterator` has the two fields
described in the following table:

| field | purpose |
| :--- | :--- |
| `next` | the index of the element to be returned by the subsequent call to the `next` method |
| `prev` | the index of the element returned by the most recent call to the `next` method;<br />value is reset to `-1` if this element is deleted by a call to `remove` |

The `next` field represents the position of the iterator as an index. If we chose not to implement `remove` then our iterator class would require only this field.

The `prev` field has two purposes. The first purpose is to store the index of the element that should be deleted if `remove` is called. The second purpose is to give
us a way to test if it is not safe to call `remove`. Recall that `remove` can be called only when two conditions hold:

1. `next` has been called at least once
    * this is needed to ensure that at least one element has been visited by the iterator
2. `remove` has been called only once for each call to `next`

A new iterator starting at the front of the list has `this.next = 0` and `this.prev = -1`. Whenever we remove an element using the iterator, we
set `this.next = -1` (because the element returned by the most recent call to `next` has now been removed from the list). Therefore, we can test
if it is not safe to call `remove` simply by testing if `this.prev == -1`.

#### Constructor

The no-argument constructor initializes an iterator that starts at the front of the list. It simply sets `this.next = 0` and `this.prev = -1`.

#### `hasNext`

`hasNext` returns `true` as long as `this.next` is a valid index for the list.

#### `next`

`next` should test if `hasNext` returns `false` and throw a `NoSuchElementException` if this is the case (indicating that there are no more elements
in the list to iterate over).

If `hasNext` returns `true`, then `next` should call the list method `get(this.next)` to get the element at index `this.next`, and then update the
fields `this.prev` and `this.next` to advance the iterator one position. Finally, `next` should return the value obtained from calling `get`.

#### `remove`

`remove` should test if `this.prev == -1` is `true` and throw an `IllegalStateException` if this is the case (indicating that `next` has not been called,
or `remove` has been called more than once since the last call to `next`).

If `this.prev == -1` is `false`, then `remove` should call the list method `remove(this.prev)` to remove the element at index `this.prev`. Then,
the value of `this.next` should be decreased by 1 (to back the iterator up one position) and the value of `this.prev` set to `-1` (to
indicate that `remove` has been called once for the most recent call to `next`).

## A complete array-based list implementation

The complete `SArrayList` and its inner `ArrayIterator` class is shown below:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Arrays;
import java.util.Iterator;
import java.util.NoSuchElementException;

import ca.queensu.cs.cisc124.notes.generics.list.SList;

public class SArrayList<E> implements SList<E> {

    private final int DEFAULT_CAPACITY = 16;
    private Object[] arr;
    private int size;

    /**
     * Initializes an empty list. 
     */
    public SArrayList() {
        this.arr = new Object[DEFAULT_CAPACITY];
        this.size = 0;
    }

    @Override
    public int size() {
        return this.size;
    }

    @Override
    public void add(E elem) {
        // do we need to resize the array?
        if (this.size() == this.arr.length) {
            this.arr = Arrays.copyOf(this.arr, this.arr.length * 2);
        }
        this.arr[this.size] = elem;
        this.size++;
    }

    /**
     * Throws an {@code IndexOutOfBoundsException} if index is less than 0 or
     * greater than {@code this.size - 1}.
     * 
     * @param index an index to validate
     * @throws {@code IndexOutOfBoundsException} if index is less than 0 or
     * greater than {@code this.size - 1}
     */
    private void checkIndex(int index) {
        if (index < 0) {
            throw new IndexOutOfBoundsException("negative index: " + index);
        } else if (index >= this.size) {
            throw new IndexOutOfBoundsException("index out of bounds: " + index + ", size: " + this.size);
        }
    }

    @Override
    public E get(int index) {
        this.checkIndex(index);
        return (E) this.arr[index];
    }

    @Override
    public E set(int index, E elem) {
        // get element at index, this also checks the index
        E old = this.get(index);
        this.arr[index] = elem;
        return old;
    }

    @Override
    public void add(int index, E elem) {
        // index can be equal to this.size
        if (index == this.size) {
            this.add(elem);
            return;
        }
        this.checkIndex(index);

        // move elements at indexes size-1, size-2, ..., index one position to right
        // i is in the index of the element to move to the right
        for (int i = this.size - 1; i >= index; i--) {
            this.arr[i + 1] = this.arr[i];                      // can this throw?
        }
        // insert elem and update size
        this.arr[index] = elem;
        this.size++;
    }

    @Override
    public E remove(int index) {
        // get element at index, this also checks the index
        E removed = this.get(index);
        
        // move elements at indexes index+1, index+2, ..., size-1 one position to left
        // i is in the index of the element to move to the right
        for (int i = index + 1; i < this.size; i++) {
            this.arr[i - 1] = this.arr[i];                      // can this throw?
        }
        // null out old last element and update size
        this.arr[this.size - 1] = null;
        this.size--;
        return removed;
    }
    
    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("[");
        if (!this.isEmpty()) {
            b.append(this.get(0));
            for (int i = 1; i < this.size; i++) {
                b.append(", ");
                b.append(this.get(i));
            }
        }
        b.append("]");
        return b.toString();
    }

    @Override
    public Iterator<E> iterator() {
        return new ArrayIterator();
    }

    private class ArrayIterator implements Iterator<E> {
        /**
         * Index of element to be returned by subsequent call to next.
         */
        private int next;
        
        /**
         * Index of element returned by most recent call to next.
         * Reset to -1 if this element is deleted by a call to remove.
         */
        private int prev;
        
        ArrayIterator() {
            this.next = 0;
            this.prev = -1;
        }
        
        @Override
        public boolean hasNext() {
            return this.next < SArrayList.this.size;
        }
        
        @Override
        public E next() {
            if (!this.hasNext()) {
                throw new NoSuchElementException();
            }
            this.prev = this.next;
            E e = SArrayList.this.get(this.next);
            this.next++;
            return e;
        }
        
        @Override
        public void remove() {
            if (this.prev == -1) {
                throw new IllegalStateException();
            }
            SArrayList.this.remove(this.prev);
            this.next--;
            this.prev = -1;
        }
    }

}

A small test program that uses an iterator to filter out the negative values from a list is shown below:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Iterator;

import ca.queensu.cs.cisc124.notes.generics.list.SList;

SList<Integer> t = new SArrayList<>();
for (int i = -5; i <= 6; i++) {
    t.add(i);
}
System.out.println(t);

for (Iterator<Integer> iter = t.iterator(); iter.hasNext(); ) {
    Integer val = iter.next();
    if (val < 0) {
        iter.remove();    // can call remove once for each call to next
    }
}
System.out.println(t);

**Exercise 1** It can be useful to create an iterator that starts at a specified index in the list instead of always starting at the beginning of the list.
Add a method to the `SArrayList` class that returns an iterator that starts at a specified index of the list, and add a constructor to the `ArrayIterator`
class that initializes an iterator so that it starts at a specified index of the list.

**Exercise 2** The largest array that can be created in Java has capacity `Integer.MAX_VALUE`. Inside the `next` method, the value
of `this.next` is always incremented by 1. Is it possible for the value of `this.next` to wrap around to `Integer.MIN_VALUE` (thus
breaking the `hasNext` method)?

**Exercise 3** (Somewhat involved; no solution is provided) Java's [ListIterator](https://docs.oracle.com/javase/8/docs/api/java/util/ListIterator.html)
interface defines an iterator over a list that can travel in both directions (forwards and backwards through the list). Modify the `ArrayIterator`
class so that implements the `ListIterator` interface.